In [1]:
from Liputan6 import Liputan6 #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [16]:
# health
obj = Liputan6()
end_date = date(2018, 8, 8)
start_date = date(2018, 1, 30)
init_page = 1
list_cat = [['news', 'news'], ['otomotif', 'otomotif'], ['properti', 'properti'],['bisnis','bisnis'],['showbiz','showbiz'],
           ['bola','bola'],['tekno','tekno'],['fashion-beauty','fashion-beauty'],['lifestye','lifestye']]
init_cat = list_cat[0:]

In [18]:
for single_date in daterange(start_date, end_date):
    for cat in init_cat:
        clear_output()
        print(single_date.strftime("%Y-%m-%d"))
        print(cat[0])
        obj.getAllBerita([], init_page, cat[0], cat[1], single_date.strftime("%Y/%m/%d"))
        init_page = 1
    init_cat = list_cat

2018-01-30
tekno
page  1
https://www.liputan6.com/tekno/indeks/2018/01/30?page=1
Insert berita  Trik Jitu Abadikan Momen Gerhana Bulan Langka Pakai Smartphone
masuk
Insert berita  Dapat Rp 16 Triliun dari Google dkk, Go-Jek Mau Pakai Buat Apa?
masuk
Insert berita  Pilkada 2018 Diprediksi Dongkrak Bisnis Printer
masuk
Insert berita  Tiap Hari, 1,3 Juta Pengguna Registrasi Kartu SIM Prabayar
masuk
Insert berita  Mi Mix 2S Jadi Smartphone Pertama yang Diperkuat Snapdragon 845?
masuk
Insert berita  Terciduk, Selebgram Ini Nekat Curi dan Edit Foto Liburan Mewah
masuk
Insert berita  Terus Berbenah, Facebook Akan Fokus Tampilkan Berita Lokal
masuk
Insert berita  Cetak Dokumen Makin Efisien dengan 4 Printer Inkjet Terbaru Canon
masuk
Insert berita  Android Oreo Beta untuk Nokia 3 Segera Hadir
masuk
Insert berita  USO Jadi Contoh untuk Hubungkan 3,5 Miliar Orang di Internet
masuk
Insert berita  Nilai Bitcoin Ternyata Berbanding Lurus dengan Pencarian Google
masuk
Insert berita  Dongkrak Jumlah 

ConnectionError: HTTPSConnectionPool(host='www.liputan6.com', port=443): Max retries exceeded with url: /tekno/read/3243027/gerhana-super-blue-blood-moon-berlangsung-besok-apa-istimewanya (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000013FB29088D0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))